In [1]:
import requests
import dlt
from dlt.sources.helpers import requests
import os
from dotenv import load_dotenv
import duckdb
import html2text
import re 
from boilerpy3 import extractors
from bs4 import BeautifulSoup

load_dotenv(override=True)

def get_stackexchange_data(tag=None,site='philosophy',page=1,pagesize=100):
  """Fetches data from the Stack Exchange API.
  
  Args:
    tag: The tag to search for.
    site: The Stack Exchange site (default: 'stackoverflow').
    page: The page number.
    pagesize: The number of items per page (max 100).

  Returns:
    A dictionary containing the API response.
  """

  url = "https://api.stackexchange.com/2.3/questions"
  # API to get the answers for a certain question with question id
  # https://api.stackexchange.com/2.3/questions/108037/answers?site=stackoverflow&filter=withbody
  params = {
    "site": site,
    "order": "asc",
    "sort": "creation",
    # "tagged": tag,
    "page": page,
    "pagesize": pagesize,
    "key": os.getenv("SE_API_KEY"),
    "filter": "withbody"
    
  }
  response = requests.get(url,params=params)
  return response.json()

# data = get_stackexchange_data()
# print(data)

# Data Ingestion

In [2]:
@dlt.resource(table_name="philosophy_questions", write_disposition="append")
def get_questions(
    tag=None, 
    site='philosophy',
    page=1,
    pagesize=100
):
    
    url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "site": site,
        "order": "asc",
        "sort": "creation",
        # "tagged": tag,
        "page": page,
        "pagesize": pagesize,
        "key": os.getenv("SE_API_KEY"),
        "filter": "withbody"        
    }

    while True:
        response = requests.get(url,params=params)
        response.raise_for_status()
        yield response.json()

        if response.json()["has_more"]==False:
            break

        if params["page"]==300:
            break
        
        params["page"]+=1


In [3]:
pipeline = dlt.pipeline(
    pipeline_name="philosophy_questions_incremental",
    destination="duckdb",
    dataset_name="philosophy_questions",
)

# load_info = pipeline.run(get_questions())
# row_counts = pipeline.last_trace.last_normalize_info

# print(row_counts)
# print("------")
# print(load_info)

In [4]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────────────────────────────────────────────────────────────┐
│                                    name                                     │
│                                   varchar                                   │
├─────────────────────────────────────────────────────────────────────────────┤
│ _dlt_loads                                                                  │
│ _dlt_pipeline_state                                                         │
│ _dlt_version                                                                │
│ philosophy_questions                                                        │
│ philosophy_questions__items                                                 │
│ philosophy_questions__items__migrated_from__other_site__aliases             │
│ philosophy_questions__items__migrated_from__other_site__markdown_extensions │
│ philosophy_questions__items__migrated_from__other_site__related_sites       │
│ philosophy_questions__items__tags     

In [5]:
questions = conn.sql("SELECT * FROM philosophy_questions__items").df()
display(questions)

,owner__account_id,owner__reputation,owner__user_id,owner__user_type,owner__profile_image,owner__display_name,owner__link,is_answered,view_count,answer_count,...,migrated_from__other_site__icon_url,migrated_from__other_site__audience,migrated_from__other_site__site_url,migrated_from__other_site__api_site_parameter,migrated_from__other_site__logo_url,migrated_from__other_site__name,migrated_from__other_site__site_type,migrated_from__on_date,migrated_from__question_id,community_owned_date
0,16491829.0,18695.0,40843.0,registered,https://graph.facebook.com/10157462114367910/p...,Kristian Berry,https://philosophy.stackexchange.com/users/408...,True,23,1,...,None,None,None,None,None,None,None,NaN,NaN,NaN
1,29736116.0,2914.0,68482.0,registered,https://i.sstatic.net/yOM8W.jpg?s=256,Meanach,https://philosophy.stackexchange.com/users/684...,True,103,4,...,None,None,None,None,None,None,None,NaN,NaN,NaN
2,5253885.0,335.0,47770.0,registered,https://www.gravatar.com/avatar/c2570aa785c372...,user,https://philosophy.stackexchange.com/users/477...,False,149,2,...,None,None,None,None,None,None,None,NaN,NaN,NaN
3,NaN,NaN,NaN,does_not_exist,None,user62907,None,True,344,8,...,None,None,None,None,None,None,None,NaN,NaN,NaN
4,19780752.0,1507.0,77058.0,registered,https://www.gravatar.com/avatar/dce3e1a595d736...,user77058,https://philosophy.stackexchange.com/users/770...,True,210,1,...,None,None,None,None,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,59775.0,2067.0,72.0,registered,https://www.gravatar.com/avatar/487b39e86dd577...,Jez,https://philosophy.stackexchange.com/users/72/jez,True,3315,1,...,None,None,None,None,None,None,None,NaN,NaN,NaN
22744,32433.0,9670.0,81.0,moderator,https://www.gravatar.com/avatar/c4752b73a675b7...,Joseph Weissman,https://philosophy.stackexchange.com/users/81/...,True,3255,2,...,None,None,None,None,None,None,None,NaN,NaN,NaN
22745,157369.0,887.0,13.0,registered,https://i.sstatic.net/LY3kx.jpg?s=256,mfg,https://philosophy.stackexchange.com/users/13/mfg,True,10332,1,...,None,None,None,None,None,None,None,NaN,NaN,NaN
22746,14250.0,375.0,102.0,registered,https://www.gravatar.com/avatar/deee1379ed13c9...,DuckMaestro,https://philosophy.stackexchange.com/users/102...,True,1158,1,...,None,None,None,None,None,None,None,NaN,NaN,NaN


In [6]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22748 entries, 0 to 22747
Data columns (total 47 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   owner__account_id                                         20153 non-null  float64
 1   owner__reputation                                         20153 non-null  float64
 2   owner__user_id                                            20153 non-null  float64
 3   owner__user_type                                          22748 non-null  object 
 4   owner__profile_image                                      20153 non-null  object 
 5   owner__display_name                                       22748 non-null  object 
 6   owner__link                                               20153 non-null  object 
 7   is_answered                                               22748 non-null  bool   
 8   view_count      

In [7]:
# CREATE A NEW TABLE WITH THE RowNum TO COUNT DUPLICATE
# conn.sql(
#     """
#     CREATE TABLE stackoverflow_questions_items_with_duplicates_count AS (
#     SELECT *, ROW_NUMBER() OVER (PARTITION BY body ORDER BY creation_date) AS RowNum
#     FROM stackoverflow_questions__items 
#     ) 
#     """
# )

# DELETE THE DUPLICATE ENTRIES FROM THE TABLE
# conn.sql(
#     """
#     DELETE FROM stackoverflow_questions_items_with_duplicates_count
#     WHERE RowNum > 1
#     """
# )


##### Get the Data for Answered Questions

In [8]:
answered_questions = questions[questions["is_answered"]==True]

In [9]:
answered_questions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19285 entries, 0 to 22747
Data columns (total 47 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   owner__account_id                                         17080 non-null  float64
 1   owner__reputation                                         17080 non-null  float64
 2   owner__user_id                                            17080 non-null  float64
 3   owner__user_type                                          19285 non-null  object 
 4   owner__profile_image                                      17080 non-null  object 
 5   owner__display_name                                       19285 non-null  object 
 6   owner__link                                               17080 non-null  object 
 7   is_answered                                               19285 non-null  bool   
 8   view_count           

In [10]:
answered_questions.head()

,owner__account_id,owner__reputation,owner__user_id,owner__user_type,owner__profile_image,owner__display_name,owner__link,is_answered,view_count,answer_count,...,migrated_from__other_site__icon_url,migrated_from__other_site__audience,migrated_from__other_site__site_url,migrated_from__other_site__api_site_parameter,migrated_from__other_site__logo_url,migrated_from__other_site__name,migrated_from__other_site__site_type,migrated_from__on_date,migrated_from__question_id,community_owned_date
0,16491829.0,18695.0,40843.0,registered,https://graph.facebook.com/10157462114367910/p...,Kristian Berry,https://philosophy.stackexchange.com/users/408...,True,23,1,...,None,None,None,None,None,None,None,NaN,NaN,NaN
1,29736116.0,2914.0,68482.0,registered,https://i.sstatic.net/yOM8W.jpg?s=256,Meanach,https://philosophy.stackexchange.com/users/684...,True,103,4,...,None,None,None,None,None,None,None,NaN,NaN,NaN
3,NaN,NaN,NaN,does_not_exist,None,user62907,None,True,344,8,...,None,None,None,None,None,None,None,NaN,NaN,NaN
4,19780752.0,1507.0,77058.0,registered,https://www.gravatar.com/avatar/dce3e1a595d736...,user77058,https://philosophy.stackexchange.com/users/770...,True,210,1,...,None,None,None,None,None,None,None,NaN,NaN,NaN
6,17436487.0,111.0,43600.0,registered,https://graph.facebook.com/117598566407271/pic...,Zhang Hong,https://philosophy.stackexchange.com/users/436...,True,4754,8,...,None,None,None,None,None,None,None,NaN,NaN,NaN


##### Check for Duplicate Entries

In [11]:
answered_questions.duplicated().sum()

0

In [12]:
answered_questions = answered_questions.iloc[:2000]

In [13]:
@dlt.resource(table_name="philosophy_answers", write_disposition="append")
def get_answers(
    tag=None, 
    site='philosophy',
    page=1,
    pagesize=100,
    questions_id=list(answered_questions["question_id"])
):
    
    for i in range(len(questions_id)):
        url = "https://api.stackexchange.com/2.3/questions" + f"/{questions_id[i]}" + "/answers" 
        params = {
            "site": site,
            # "tagged": tag,
            "page": page,
            "pagesize": pagesize,
            "key": os.getenv("SE_API_KEY"),
            "filter": "withbody"        
        }

        while True:
            response = requests.get(url,params=params)
            response.raise_for_status()
            yield response.json()

            if response.json()["has_more"]==False:
                break
            
            params["page"]+=1


In [14]:
pipeline = dlt.pipeline(
    pipeline_name="philosophy_answers_incremental",
    destination="duckdb",
    dataset_name="philosophy_answers",
)

# load_info = pipeline.run(get_answers())
# row_counts = pipeline.last_trace.last_normalize_info

# print(row_counts)
# print("------")
# print(load_info)

In [15]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌───────────────────────────┐
│           name            │
│          varchar          │
├───────────────────────────┤
│ _dlt_loads                │
│ _dlt_pipeline_state       │
│ _dlt_version              │
│ philosophy_answers        │
│ philosophy_answers__items │
└───────────────────────────┘

In [22]:
answers = conn.sql("SELECT * FROM philosophy_answers__items").df()
display(answers)

,owner__account_id,owner__reputation,owner__user_id,owner__user_type,owner__profile_image,owner__display_name,owner__link,is_accepted,score,last_activity_date,...,answer_id,question_id,content_license,body,_dlt_parent_id,_dlt_list_idx,_dlt_id,last_edit_date,owner__accept_rate,community_owned_date
0,16252558.0,31306.0,40730.0,registered,https://www.gravatar.com/avatar/b4e30985026dd7...,J D,https://philosophy.stackexchange.com/users/407...,False,3,1725632358,...,116886,116884,CC BY-SA 4.0,<p><em>You used a question mark as a superscri...,wnCoJ+8xcSGfAg,0,4ZfPEQTz3J4Q+A,NaN,NaN,NaN
1,4556922.0,4291.0,37256.0,registered,https://www.gravatar.com/avatar/cad58aa1de61af...,Rushi,https://philosophy.stackexchange.com/users/372...,False,1,1725631539,...,116881,116834,CC BY-SA 4.0,<p>There have been a number of philosophers of...,2/+CVrJnVEuYfg,0,OcVSPXuYlR7/qg,1.725632e+09,NaN,NaN
2,29116279.0,2450.0,67046.0,registered,https://www.gravatar.com/avatar/d7e7397a342ef4...,Olivier5,https://philosophy.stackexchange.com/users/670...,False,2,1725624713,...,116882,116834,CC BY-SA 4.0,<p>Lots of great answers and suggestions. Let ...,2/+CVrJnVEuYfg,1,I3/Pz9DCXCqrhA,NaN,NaN,NaN
3,29736116.0,2914.0,68482.0,registered,https://i.sstatic.net/yOM8W.jpg?s=256,Meanach,https://philosophy.stackexchange.com/users/684...,False,2,1725610565,...,116878,116834,CC BY-SA 4.0,"<p>I paraphrase Churchill, but this version co...",2/+CVrJnVEuYfg,2,khn6NuV1YSHUKQ,NaN,NaN,NaN
4,4896115.0,36748.0,9174.0,registered,https://i.sstatic.net/ddQui.jpg?s=256,Jo Wehler,https://philosophy.stackexchange.com/users/917...,False,1,1725464830,...,116839,116834,CC BY-SA 4.0,<ol>\n<li><p>Your question starts with a presu...,2/+CVrJnVEuYfg,3,NY1XJjqF0b+W8w,NaN,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8192,12482509.0,11027.0,50852.0,registered,https://www.gravatar.com/avatar/488ae937ea41c0...,David Gudeman,https://philosophy.stackexchange.com/users/508...,False,3,1683860487,...,99227,99225,CC BY-SA 4.0,<p>The clue is in the fact that he called it <...,69f5IZd441aYLw,2,4b5rjXngZt/o7g,NaN,NaN,NaN
8193,4896115.0,36748.0,9174.0,registered,https://i.sstatic.net/ddQui.jpg?s=256,Jo Wehler,https://philosophy.stackexchange.com/users/917...,False,1,1703321679,...,106655,106274,CC BY-SA 4.0,<p>Following the suggestion of @MauroALLEGRANZ...,opfDfei1nTH+iQ,0,+LPgvTAcfPWL1A,1.703322e+09,62.0,NaN
8194,12141261.0,15091.0,29339.0,registered,https://i.sstatic.net/0VofS.jpg?s=256,Dcleve,https://philosophy.stackexchange.com/users/293...,False,0,1702837558,...,106488,106274,CC BY-SA 4.0,"<p>The core problem that you are asking about,...",opfDfei1nTH+iQ,1,JnQ0FbklNgIruQ,NaN,NaN,NaN
8195,29736116.0,2914.0,68482.0,registered,https://i.sstatic.net/yOM8W.jpg?s=256,Meanach,https://philosophy.stackexchange.com/users/684...,False,0,1702836041,...,106485,106274,CC BY-SA 4.0,<p>I am afraid that your question describes a ...,opfDfei1nTH+iQ,2,fIqkGTSNe70vxw,NaN,NaN,NaN


In [23]:
answers["body"].iloc[0]

'<p><em>You used a question mark as a superscript to a question mark. Awesome. :D</em></p>\n<p>You say:</p>\n<blockquote>\n<p>So wouldn\'t a quantifier-free language have to eschew set theory and arithmetic to that extent, to get to where its speakers would want to go? Might that destabilize such a language even more, then, though?</p>\n</blockquote>\n<p>So, let\'s draw a very clear line in the sand. One can talk about natural languages which are a product of evolution and shared intentionality (according to Tomasello at least), or one can talk about artificial languages like those developed by Descartes, Frege, and the modern mathematical logician or programming grammar engineer. <strong>Understanding how natural languages can be more fully understood with formalisms is the central focus of linguistic formal semantics, a movement started by <a href="https://en.wikipedia.org/wiki/Richard_Montague" rel="nofollow noreferrer">Richard Monatague</a> and championed by <a href="https://en.wik